In [1]:
import pandas as pd
import numpy as np

In [2]:
import psycopg2
try:
    conn = psycopg2.connect("dbname=metacritic user=postgres")
except:
    print("cannot connect")

/Users/michael/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
cur = conn.cursor()

In [4]:
import pandas.io.sql as sqlio
sql = """select round(avg(ratings.album_rating),1) as avg_rating, count(ratings.album_rating) as cnt,
albums.artist, ratings.critic_name, ratings.critic_id from ratings
JOIN albums ON albums.id = ratings.album_id
WHERE albums.artist != 'Various Artists'
group by albums.artist, ratings.critic_name, ratings.critic_id
order by cnt desc;"""

ratings_df = sqlio.read_sql_query(sql, conn)

In [6]:
ratings_df.head()
ratings_df.shape

(118852, 5)

In [6]:
ratings_pivot = pd.pivot_table(ratings_df, index='critic_name', columns='artist', values='avg_rating',aggfunc=np.max,fill_value=0);
ratings_pivot.head()

artist                                  !!! [Chik Chik Chik]  '68  (+44)  \
critic_name                                                                
Absolute Punk (Staff reviews)                            0.0    0      0   
AllMusic                                                84.3    0     70   
Almost Cool                                             71.3    0      0   
Alternative Press                                       76.0  100     60   
Alternative Press (Record Of The Week)                   0.0    0      0   

artist                                  (Sandy) Alex G  *NSYNC  +++ (Crosses)  \
critic_name                                                                     
Absolute Punk (Staff reviews)                        0       0              0   
AllMusic                                            80       0             70   
Almost Cool                                          0       0              0   
Alternative Press                                    0       0             80   
Alternative Press (Record Of The Week)               0       0              0   

artist                                  +/-  \
critic_name                                   
Absolute Punk (Staff reviews)             0   
AllMusic                                 80   
Almost Cool                               0   
Alternative Press                        80   
Alternative Press (Record Of The Week)    0   

artist                                  ...And You Will Know Us by the Trail of Dead  \
critic_name                                                                            
Absolute Punk (Staff reviews)                                                   85.0   
AllMusic                                                                        74.3   
Almost Cool                                                                     76.5   
Alternative Press                                                               75.6   
Alternative Press (Record Of The Week)                                           0.0   

artist                                  0xCE 0xBC-Ziq  10,000 Maniacs ...  \
critic_name                                                           ...   
Absolute Punk (Staff reviews)                       0               0 ...   
AllMusic                                           70              60 ...   
Almost Cool                                         0               0 ...   
Alternative Press                                   0               0 ...   
Alternative Press (Record Of The Week)              0               0 ...   

artist                                  thenewno2  tētēma  will.i.am  \
critic_name                                                            
Absolute Punk (Staff reviews)                   0       0        0.0   
AllMusic                                        0      80       60.0   
Almost Cool                                     0       0        0.0   
Alternative Press                               0      70        0.0   
Alternative Press (Record Of The Week)          0       0        0.0   

artist                                  worriedaboutsatan  Áine O'Dwyer  \
critic_name                                                               
Absolute Punk (Staff reviews)                           0             0   
AllMusic                                                0             0   
Almost Cool                                             0             0   
Alternative Press                                       0             0   
Alternative Press (Record Of The Week)                  0             0   

artist                                  Árabrot  Ásgeir  Ólafur Arnalds  \
critic_name                                                               
Absolute Punk (Staff reviews)                 0       0               0   
AllMusic                                      0      70              60   
Almost Cool                                   0       0               0   
Altern

In [7]:
ratings_pivot.shape

(129, 7739)

In [8]:
X = ratings_pivot.values.T # transpose pivot table, getting SVD of critic preferences
X.shape

(7739, 129)

### Decomposing the Matrix

In [9]:
import sklearn
from sklearn.decomposition import TruncatedSVD

In [39]:
SVD = TruncatedSVD(n_components=15,random_state=17)
resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(7739, 15)

### Generating Correlation Matrix

In [40]:
corr_mat = np.corrcoef(resultant_matrix)

### (for PostgreSQL purposes)

In [41]:
corr_list = corr_mat.tolist()
len(corr_list)

7739

In [42]:
artist_names = ratings_pivot.columns
artist_list = list(artist_names)
len(artist_list)

7739

In [43]:
artists_df = pd.DataFrame({'artist':artist_list}); 

In [44]:
artists_df.head()

artist
0  !!! [Chik Chik Chik]
1                   '68
2                 (+44)
3        (Sandy) Alex G
4                *NSYNC

In [45]:
corr_list = corr_mat.tolist()

In [46]:
corr_df = pd.DataFrame({'corr_vector': corr_list})

In [47]:
corr_df.head()

corr_vector
0  [0.9999999999999999, 0.278365727142314, 0.5105...
1  [0.278365727142314, 0.9999999999999998, 0.2903...
2  [0.5105521658881724, 0.2903161015349604, 1.0, ...
3  [0.6676212188228522, 0.20843020749245114, 0.21...
4  [0.32267373484904016, 0.03714714327201794, 0.7...

In [48]:
postgres_df = artists_df.join(corr_df) 
postgres_df['row'] = postgres_df.index+1

In [49]:
postgres_df.head()

artist                                        corr_vector  \
0  !!! [Chik Chik Chik]  [0.9999999999999999, 0.278365727142314, 0.5105...   
1                   '68  [0.278365727142314, 0.9999999999999998, 0.2903...   
2                 (+44)  [0.5105521658881724, 0.2903161015349604, 1.0, ...   
3        (Sandy) Alex G  [0.6676212188228522, 0.20843020749245114, 0.21...   
4                *NSYNC  [0.32267373484904016, 0.03714714327201794, 0.7...   

   row  
0    1  
1    2  
2    3  
3    4  
4    5

In [63]:
postgres_df.to_csv('/Users/michael/Desktop/corr_matrix.csv',index=False)

## User Input

In [65]:
artist = input("Enter artist name: ")

Enter artist name: Radiohead


In [66]:
selected_artist = artist_list.index(artist)
selected_artist

5263

In [67]:
corr_selected_artist = corr_mat[selected_artist]
corr_selected_artist.shape

(7739,)

In [68]:
#corr_range = (corr_selected_artist < 1.0) & (corr_selected_artist > 0.9);
#suggested_artists = list(artist_names[corr_range])
id_corr_pairings = dict(zip(artist_names,corr_selected_artist))

# create data frame linking ids and album correlations so we can sort by correlation!
corr_df = pd.DataFrame(list(id_corr_pairings.items()),columns=['id','corr'])
#recommendations = corr_df.loc[(corr_df['corr'] > 0.9) & (corr_df['corr'] < 1.0)]
corr_df.sort_values(['corr'],ascending=False).head(10)

# TODO: maybe suggest album with highest MC rating for each artist?

id      corr
5263          Radiohead  1.000000
4064    Magnetic Fields  0.996137
6541   The Flaming Lips  0.993323
4860          PJ Harvey  0.991917
7490              Wilco  0.990525
5975              Spoon  0.989897
7623        Yo La Tengo  0.989759
593                Beck  0.987658
5774          Sigur Rós  0.985434
607   Belle & Sebastian  0.984228